<a href="https://colab.research.google.com/github/Brent-Morrison/Coursera_Capstone/blob/master/applied_data_science_capstone_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Applied Data Science Capstone Project - week 4**

# Introduction

This project will seek to determine if political persuasion influences uptake of government assistance.

We will investigate the uptake of loans granted under the Small Business Administration Paycheck Protection Program ("SBA-PPP") in the United States.

The SBA-PPP was set up by the US Treasury to provide funding to businesses impacted by the COVID-19 pandemic.

The outcome of this analysis will be of interest to social scientists studying the impact of political persuasion on behaviours of businesses in the United States.  

The idea for this project came after reading a number of articles pointing out inconsistencies in an organisations beliefs and actions in relation to government assistance.  These two articles ([In sign of the times, Ayn Rand Institute approved for PPP loan](https://www.reuters.com/article/us-health-coronavirus-ppp-ayn-rand/in-sign-of-the-times-ayn-rand-institute-approved-for-ppp-loan-idUSKBN248026), and [Vocal Opponents Of Federal Spending Took PPP Loans](https://www.forbes.com/sites/andrewsolender/2020/07/06/vocal-opponents-of-federal-spending-took-ppp-loans-including-ayn-rand-institute-grover-norquist-group/#7c2ad6703d53) ), highlight what the authors see as hypocritical behaviour.

These examples represent individual data points and in all likelihood are called out for their ability to generate headlines.

A more thorough analysis will use granular data, that covering a broad population of firms, to answer the question posed above.  Posing that question in a slightly different way.  Are those who we would expect to be opposed to government intervention, less likely to accept government assistance? 

# Data

Determining if political persuasion influences uptake of loans requires us to do three things:
1. Quantify loan uptake
2. Assess political persuasion
3. Identify and hold other characteristics constant

Data sources are outlined below.

### Loan uptake
Data on loans granted under the Small Business Administration Paycheck Protection Program ("SBA-PPP") has been made available at the U.S Treasury [website](https://home.treasury.gov/policy-issues/cares-act/assistance-for-small-businesses/sba-paycheck-protection-program-loan-level-data).  This data set contains records for each loan recipient along with attributes such as geographical location, industry membership and business type.

### Political persuasion
We will assess political persuasion based on electoral results data.  Our source for this data is the [MIT Election Data and Science lab](https://electionlab.mit.edu/data).  This organisation has published numerous election results datasets to [Github](https://github.com/MEDSL).

### Confounding characteristics
The characteristics held constant will be a combination of demographic and industry attributes. Demographic data will be sourced from US census data via the [uszipcode](https://uszipcode.readthedocs.io/index.html) python library.  This library aggregates geographic, demographic, employment and education data.  

Foursquare data will be used to assess the type of region or neighbourhood.  This will be done using the quantity and type of businesses in specific geographical regions.



# Data source examples

The code blocks below provide examples of the data sources mentioned above.

In [1]:
# Required libraries
import numpy as np
import pandas as pd
import requests
import time

The **SBA-PPP** data has been downloaded from the US Department of the Treasury website to my google drive folder.  This data was downloaded 15th July 2020.

In [2]:
# Mount Google Drive to Collaboratory
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
ppp_df = pd.read_csv('gdrive/My Drive/PPP_Data_150k_plus.csv')
ppp_df[ppp_df['State'] == 'IL'].head()

,LoanRange,BusinessName,Address,City,State,Zip,NAICSCode,BusinessType,RaceEthnicity,Gender,Veteran,NonProfit,JobsRetained,DateApproved,Lender,CD
212944,a $5-10 million,3D EXHIBITS INC.,ALBION AVE,SCHAUMBURG,IL,60193.0,561920.0,Corporation,Unanswered,Unanswered,Unanswered,NaN,187.0,04/05/2020,"Village Bank and Trust, National Association",IL - 08
212945,a $5-10 million,A M CASTLE AND CO,1420 KENSINGTON RD Suite 220,OAK BROOK,IL,60523.0,423510.0,Corporation,Unanswered,Unanswered,Unanswered,NaN,0.0,04/27/2020,"PNC Bank, National Association",IL - 05
212946,a $5-10 million,"AARETE, INC.",200 E. Randolph Street Suite 3010,CHICAGO,IL,60601.0,541611.0,Subchapter S Corporation,Unanswered,Unanswered,Unanswered,NaN,270.0,04/08/2020,BMO Harris Bank National Association,IL - 07
212947,a $5-10 million,"ABCOM TECHNOLOGY GROUP, INC.",1456 NORWOOD AVE,ITASCA,IL,60143.0,561920.0,Corporation,Unanswered,Unanswered,Unanswered,NaN,448.0,04/07/2020,Western Alliance Bank,IL - 08
212948,a $5-10 million,ACCURATE PERSONNEL LLC,33 S ROSELLE RD,SCHAUMBURG,IL,60193.0,561320.0,Limited Liability Company(LLC),Unanswered,Male Owned,Non-Veteran,NaN,500.0,04/08/2020,"Schaumburg Bank & Trust Company, National Asso...",IL - 08


**MIT Election Data and Science lab** github data.

In [4]:
# https://electionlab.mit.edu/data
df_county = pd.read_csv('https://raw.githubusercontent.com/MEDSL/2018-elections-official/master/county_2018.csv', encoding = 'ISO-8859-1')
df_county[df_county['state_po'] == 'IL'].head()

,year,state,state_po,state_fips,state_cen,state_ic,county,office,district,stage,special,rank,candidate,party,writein,mode,candidatevotes,totalvotes,unofficial,version
54580,2018,Illinois,IL,17,33,21,Adams,Attorney General,statewide,gen,False,NaN,Bubba Harsy,libertarian,False,total,597.0,4548409.0,False,20190430
54581,2018,Illinois,IL,17,33,21,Adams,Attorney General,statewide,gen,False,NaN,Erika Harold,republican,False,total,17910.0,4548409.0,False,20190430
54582,2018,Illinois,IL,17,33,21,Adams,Attorney General,statewide,gen,False,NaN,Kwame Raoul,democrat,False,total,5748.0,4548409.0,False,20190430
54583,2018,Illinois,IL,17,33,21,Adams,Comptroller,statewide,gen,False,NaN,Claire Ball,libertarian,False,total,730.0,4535759.0,False,20190430
54584,2018,Illinois,IL,17,33,21,Adams,Comptroller,statewide,gen,False,NaN,Darlene Senger,republican,False,total,15853.0,4535759.0,False,20190430


**Demographic data** is sourced using the [uszipcode](https://uszipcode.readthedocs.io/index.html) python library.

In [5]:
!pip install uszipcode

In [6]:
from uszipcode import SearchEngine, SimpleZipcode

In [7]:
search = SearchEngine(simple_zipcode=True)
zipcode = search.by_zipcode('60193')
zipcode

SimpleZipcode(zipcode='60193', zipcode_type='Standard', major_city='Schaumburg', post_office_city='Schaumburg, IL', common_city_list=['Schaumburg'], county='Cook County', state='IL', lat=42.01, lng=-88.09, timezone='Central', radius_in_miles=4.0, area_code_list=['224', '312', '630', '708', '847'], population=39188, population_density=4289.0, land_area_in_sqmi=9.14, water_area_in_sqmi=0.03, housing_units=16649, occupied_housing_units=16019, median_home_value=257100, median_household_income=72939, bounds_west=-88.154324, bounds_east=-88.031254, bounds_north=42.029954, bounds_south=41.988685)

**Foursquare** data for information on individual neighbourhoods.

In [8]:
# Foursquare credentials
CLIENT_ID='0M35HSSAZ1L2QPGP5LAKSPMBUUDTERESBKXLNZRQMUCEVJYS'
CLIENT_SECRET='QWIGICNR1KXB3RU5Y5LRJFH03RANUYOVRKNTUTO0KCONHAGQ'
VERSION='20200623'
LIMIT=100

In [9]:
# Function for querying Foursquare
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        #results = requests.get(url).json()["response"]['groups'][0]['items']
        # Alternate approach due to KeyError on "groups"
        results = requests.get(url).json()["response"].get('groups')[0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        
        # Prevent disconnect
        time.sleep(0.05)

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [10]:
# Example data
schaumburg = getNearbyVenues(
    names=['Schaumburg'],
    latitudes=[42.01],
    longitudes=[-88.09]
  )
schaumburg.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Schaumburg,42.01,-88.09,Access Taxi,42.01332,-88.092928,Bar
1,Schaumburg,42.01,-88.09,Sue Garcia Fitness,42.01383,-88.092864,Gym / Fitness Center
